# OTEX Quick Start

This notebook demonstrates the basic usage of OTEX for OTEC plant design and analysis.

## Contents
1. [Configuration](#1.-Configuration)
2. [Plant Sizing](#2.-Plant-Sizing)
3. [Cost Analysis](#3.-Cost-Analysis)
4. [Visualization](#4.-Visualization)

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# OTEX imports
from otex.config import parameters_and_constants
from otex.plant.sizing import otec_sizing
from otex.economics.costs import capex_opex_lcoe

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## 1. Configuration

Set up the OTEC plant parameters using `parameters_and_constants()`.

In [ ]:
# Configure a 50 MW OTEC plant
inputs = parameters_and_constants(
    p_gross=-50000,           # 50 MW gross power (negative = output)
    cost_level='low_cost',    # Optimistic cost assumptions
    cycle_type='rankine_closed',
    fluid_type='ammonia',
    year=2020
)

# Display key parameters
print("OTEC Plant Configuration")
print("=" * 40)
print(f"Gross Power:      {-inputs['p_gross']/1000:.0f} MW")
print(f"Cycle Type:       {inputs['cycle_type']}")
print(f"Working Fluid:    {inputs['fluid_type']}")
print(f"Cost Level:       {inputs['cost_level']}")
print(f"\nEconomic Parameters")
print(f"Discount Rate:    {inputs['discount_rate']:.1%}")
print(f"Plant Lifetime:   {inputs['lifetime']} years")
print(f"Capacity Factor:  {inputs['availability_factor']:.1%}")
print(f"CRF:              {inputs['crf']:.4f}")

## 2. Plant Sizing

Size the OTEC plant for specific ocean temperature conditions.

In [ ]:
# Define ocean conditions (typical Caribbean)
T_WW = np.array([28.0])  # Warm water temperature (°C)
T_CW = np.array([5.0])   # Cold water temperature (°C)

# Temperature drops across heat exchangers
del_T_WW = 3.0  # °C - warm water temperature drop
del_T_CW = 3.0  # °C - cold water temperature rise

print(f"Ocean Conditions")
print(f"=" * 40)
print(f"Warm water (surface):  {T_WW[0]:.1f} °C")
print(f"Cold water (1000m):    {T_CW[0]:.1f} °C")
print(f"Temperature difference: {T_WW[0] - T_CW[0]:.1f} °C")

In [ ]:
# Size the plant
plant = otec_sizing(T_WW, T_CW, del_T_WW, del_T_CW, inputs, 'low_cost')

# Display results
print("Plant Sizing Results")
print("=" * 40)
print(f"\nPower Output")
print(f"  Gross Power:     {-inputs['p_gross']/1000:.1f} MW")
print(f"  Net Power:       {-plant['p_net_nom'][0]/1000:.1f} MW")
print(f"  Parasitic Load:  {(-inputs['p_gross'] + plant['p_net_nom'][0])/1000:.1f} MW")
print(f"  Net Efficiency:  {plant['eff_net_nom'][0]*100:.2f}%")

print(f"\nHeat Exchangers")
print(f"  Evaporator Area: {plant['A_evap'][0]:,.0f} m²")
print(f"  Condenser Area:  {plant['A_cond'][0]:,.0f} m²")
print(f"  Total HX Area:   {(plant['A_evap'][0] + plant['A_cond'][0]):,.0f} m²")

print(f"\nSeawater Flows")
print(f"  Warm Water:      {plant['m_WW_nom'][0]:,.0f} kg/s")
print(f"  Cold Water:      {plant['m_CW_nom'][0]:,.0f} kg/s")
print(f"  Total Flow:      {(plant['m_WW_nom'][0] + plant['m_CW_nom'][0]):,.0f} kg/s")

## 3. Cost Analysis

Calculate CAPEX, OPEX, and LCOE.

In [ ]:
# Set distance to shore (affects cable and transmission costs)
dist_shore = 20.0  # km
inputs['dist_shore'] = np.array([dist_shore])

# Calculate transmission efficiency
# AC for <= 50 km, DC for > 50 km
if dist_shore <= 50:
    inputs['eff_trans'] = 0.979 - 1e-6 * dist_shore**2 - 9e-5 * dist_shore
else:
    inputs['eff_trans'] = 0.964 - 8e-5 * dist_shore

print(f"Distance to shore: {dist_shore:.0f} km")
print(f"Transmission efficiency: {inputs['eff_trans']:.1%}")

In [ ]:
# Calculate costs
costs_dict, capex_total, opex, lcoe = capex_opex_lcoe(plant, inputs, 'low_cost')

print("Cost Analysis Results")
print("=" * 40)
print(f"\nTotal Costs")
print(f"  CAPEX:  ${capex_total[0]/1e6:,.1f} M")
print(f"  OPEX:   ${opex[0]/1e6:,.1f} M/year")
print(f"  LCOE:   {lcoe[0]:.2f} ct/kWh")

print(f"\nCAPEX Breakdown")
print(f"  Turbine:         ${costs_dict['turbine_CAPEX']/1e6:,.1f} M")
print(f"  Evaporator:      ${costs_dict['evap_CAPEX']/1e6:,.1f} M")
print(f"  Condenser:       ${costs_dict['cond_CAPEX']/1e6:,.1f} M")
print(f"  Pumps:           ${costs_dict['pump_CAPEX']/1e6:,.1f} M")
print(f"  Pipes:           ${costs_dict['pipes_CAPEX']/1e6:,.1f} M")
print(f"  Platform:        ${costs_dict['platform_CAPEX']/1e6:,.1f} M")
print(f"  Mooring:         ${costs_dict['mooring_CAPEX']/1e6:,.1f} M")
print(f"  Cables:          ${costs_dict['cable_CAPEX']/1e6:,.1f} M")
print(f"  Deployment:      ${costs_dict['deploy_CAPEX']/1e6:,.1f} M")
print(f"  Controls:        ${costs_dict['man_CAPEX']/1e6:,.1f} M")
print(f"  Contingency:     ${costs_dict['extra_CAPEX']/1e6:,.1f} M")

## 4. Visualization

In [ ]:
# CAPEX breakdown pie chart
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
labels = ['Turbine', 'Heat Exchangers', 'Pumps', 'Pipes', 
          'Platform', 'Mooring', 'Cables', 'Other']
sizes = [
    costs_dict['turbine_CAPEX'],
    costs_dict['evap_CAPEX'] + costs_dict['cond_CAPEX'],
    costs_dict['pump_CAPEX'],
    costs_dict['pipes_CAPEX'],
    costs_dict['platform_CAPEX'],
    costs_dict['mooring_CAPEX'],
    costs_dict['cable_CAPEX'],
    costs_dict['deploy_CAPEX'] + costs_dict['man_CAPEX'] + costs_dict['extra_CAPEX']
]

colors = plt.cm.Set3(np.linspace(0, 1, len(labels)))
axes[0].pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90)
axes[0].set_title('CAPEX Breakdown', fontsize=12, fontweight='bold')

# Bar chart
categories = ['Turbine', 'Evap', 'Cond', 'Pumps', 'Pipes', 
              'Platform', 'Mooring', 'Cables', 'Deploy', 'Controls']
values = [
    costs_dict['turbine_CAPEX']/1e6,
    costs_dict['evap_CAPEX']/1e6,
    costs_dict['cond_CAPEX']/1e6,
    costs_dict['pump_CAPEX']/1e6,
    costs_dict['pipes_CAPEX']/1e6,
    costs_dict['platform_CAPEX']/1e6,
    costs_dict['mooring_CAPEX']/1e6,
    costs_dict['cable_CAPEX']/1e6,
    costs_dict['deploy_CAPEX']/1e6,
    costs_dict['man_CAPEX']/1e6
]

bars = axes[1].bar(categories, values, color='steelblue', edgecolor='white')
axes[1].set_ylabel('Cost ($ Million)', fontsize=11)
axes[1].set_title('CAPEX by Component', fontsize=12, fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)

# Add value labels
for bar, val in zip(bars, values):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'${val:.0f}M', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('capex_breakdown.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Sensitivity to Temperature Difference

Analyze how LCOE changes with different temperature conditions.

In [ ]:
# Vary temperature difference
T_WW_range = np.arange(25, 31, 0.5)
T_CW_fixed = 5.0

results = []
for T_WW_val in T_WW_range:
    T_WW_arr = np.array([T_WW_val])
    T_CW_arr = np.array([T_CW_fixed])
    
    try:
        plant_temp = otec_sizing(T_WW_arr, T_CW_arr, del_T_WW, del_T_CW, inputs, 'low_cost')
        _, _, _, lcoe_temp = capex_opex_lcoe(plant_temp, inputs, 'low_cost')
        
        results.append({
            'T_WW': T_WW_val,
            'delta_T': T_WW_val - T_CW_fixed,
            'net_power': -plant_temp['p_net_nom'][0]/1000,
            'efficiency': plant_temp['eff_net_nom'][0]*100,
            'lcoe': lcoe_temp[0]
        })
    except:
        pass

df_results = pd.DataFrame(results)
df_results

In [ ]:
# Plot sensitivity
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# LCOE vs Temperature Difference
axes[0].plot(df_results['delta_T'], df_results['lcoe'], 'o-', color='crimson', linewidth=2)
axes[0].set_xlabel('Temperature Difference (°C)', fontsize=11)
axes[0].set_ylabel('LCOE (ct/kWh)', fontsize=11)
axes[0].set_title('LCOE vs Temperature Difference', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Net Power vs Temperature Difference
axes[1].plot(df_results['delta_T'], df_results['net_power'], 's-', color='forestgreen', linewidth=2)
axes[1].set_xlabel('Temperature Difference (°C)', fontsize=11)
axes[1].set_ylabel('Net Power (MW)', fontsize=11)
axes[1].set_title('Net Power vs Temperature Difference', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Efficiency vs Temperature Difference
axes[2].plot(df_results['delta_T'], df_results['efficiency'], '^-', color='royalblue', linewidth=2)
axes[2].set_xlabel('Temperature Difference (°C)', fontsize=11)
axes[2].set_ylabel('Net Efficiency (%)', fontsize=11)
axes[2].set_title('Efficiency vs Temperature Difference', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('temperature_sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()

## Summary

This notebook demonstrated:

1. **Configuration**: Setting up plant parameters with `parameters_and_constants()`
2. **Plant Sizing**: Sizing components with `otec_sizing()`
3. **Cost Analysis**: Calculating CAPEX, OPEX, LCOE with `capex_opex_lcoe()`
4. **Visualization**: Creating charts for cost breakdown
5. **Sensitivity**: Analyzing impact of temperature difference

### Key Findings

- Higher temperature difference → Lower LCOE
- Heat exchangers and platform are major cost components
- Net efficiency is typically 2-3%

### Next Steps

- [Regional Analysis](02_regional_analysis.ipynb) - Analyze specific locations
- [Uncertainty Analysis](03_uncertainty_analysis.ipynb) - Quantify uncertainty